In [1]:
from neo4j import GraphDatabase, basic_auth
import csv

In [2]:
driver = GraphDatabase.driver(
  "bolt://3.238.143.211:7687",
  auth=basic_auth("neo4j", "checker-shift-chips"))

In [3]:
patients_csv_data = []
with open("csv/patients.csv", newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        patients_csv_data.append(row)

In [4]:
patients_csv_data[0]

{'Id': '30a6452c-4297-a1ac-977a-6a23237c7b46',
 'BIRTHDATE': '1994-02-06',
 'DEATHDATE': '',
 'SSN': '999-52-8591',
 'DRIVERS': 'S99996852',
 'PASSPORT': 'X47758697X',
 'PREFIX': 'Mr.',
 'FIRST': 'Joshua658',
 'MIDDLE': 'Alvin56',
 'LAST': 'Kunde533',
 'SUFFIX': '',
 'MAIDEN': '',
 'MARITAL': 'M',
 'RACE': 'white',
 'ETHNICITY': 'nonhispanic',
 'GENDER': 'M',
 'BIRTHPLACE': 'Boston  Massachusetts  US',
 'ADDRESS': '811 Kihn Viaduct',
 'CITY': 'Braintree',
 'STATE': 'Massachusetts',
 'COUNTY': 'Norfolk County',
 'FIPS': '25021',
 'ZIP': '02184',
 'LAT': '42.21114202874998',
 'LON': '-71.0458021760648',
 'HEALTHCARE_EXPENSES': '56904.96',
 'HEALTHCARE_COVERAGE': '18019.99',
 'INCOME': '100511'}

In [5]:
def upload_patients(tx, records):
    cypher_query = """
    UNWIND $records as record
    MERGE(p:Patient {Id: record.Id})
    SET p += record
    """
    tx.run(cypher_query, records=records)

with driver.session(database="neo4j") as session:
    session.execute_write(upload_patients, patients_csv_data)

In [6]:
conditions_csv_data = []
with open("csv/conditions.csv", newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        conditions_csv_data.append(row)

In [7]:
def upload_conditions(tx, records):
    cypher_query = """
    UNWIND $records as record
    CREATE (c:Condition)
    SET c = record
    WITH c, record
    MATCH (p:Patient {Id: record.PATIENT})
    MERGE (p)-[:HAS_CONDITION]->(c)
    """
    tx.run(cypher_query, records=records)

with driver.session(database="neo4j") as session:
    session.execute_write(upload_conditions, conditions_csv_data)

In [8]:
encounters_csv_data = []
with open("csv/encounters.csv", newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        encounters_csv_data.append(row)

In [9]:
def upload_encounters(tx, records):
    cypher_query = """
    UNWIND $records as record
    MERGE (e:Encounter {Id: record.Id})
    SET e = record
    WITH e, record
    MATCH (p:Patient {Id: record.PATIENT})
    MERGE (p)-[:HAS_ENCOUNTER]->(e)
    WITH e, record
    MATCH (c:Condition {ENCOUNTER: record.Id})
    MERGE (c)-[:FROM_ENCOUNTER]->(e)
    """
    tx.run(cypher_query, records=records)

with driver.session(database="neo4j") as session:
    session.execute_write(upload_encounters, encounters_csv_data)

In [10]:
observations_csv_data = []
with open("csv/observations.csv", newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        observations_csv_data.append(row)

In [11]:
def upload_observations(tx, records):
    cypher_query = """
    UNWIND $records as record
    CREATE (o:Observation)
    SET o = record
    WITH o, record
    MATCH (e:Encounter {Id: record.ENCOUNTER})
    MERGE (e)-[:HAS_OBSERVATION]->(o)
    """
    tx.run(cypher_query, records=records)

with driver.session(database="neo4j") as session:
    session.execute_write(upload_observations, observations_csv_data)

In [6]:
organizations_csv_data = []
with open("csv/organizations.csv", newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        organizations_csv_data.append(row)

In [7]:
def upload_organizations(tx, records):
    cypher_query = """
    UNWIND $records as record
    MERGE (org:Organization {Id: record.Id})
    SET org += record
    WITH org, record
    MATCH (e:Encounter {ORGANIZATION: record.Id})
    MERGE (e)-[:WITH_ORGANIZATION]->(org)
    """
    tx.run(cypher_query, records=records)

with driver.session(database="neo4j") as session:
    session.execute_write(upload_organizations, organizations_csv_data)

In [8]:
medications_csv_data = []
with open("csv/medications.csv", newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        medications_csv_data.append(row)

In [9]:
def upload_medications(tx, records):
    cypher_query = """
    UNWIND $records as record
    CREATE (m:Medication)
    SET m = record
    WITH m, record
    MATCH (e:Encounter {Id: record.ENCOUNTER})
    MERGE (m)-[:FROM_ENCOUNTER]->(e)
    """
    tx.run(cypher_query, records=records)

with driver.session(database="neo4j") as session:
    session.execute_write(upload_medications, medications_csv_data)

In [10]:
providers_csv_data = []
with open("csv/providers.csv", newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        providers_csv_data.append(row)

In [11]:
def upload_providers(tx, records):
    cypher_query = """
    UNWIND $records as record
    MERGE (pro:Provider {Id: record.Id})
    SET pro += record
    WITH pro, record
    MATCH (org:Organization {Id: record.ORGANIZATION})
    MERGE (pro)-[:WORKS_AT]->(org)
    """
    tx.run(cypher_query, records=records)

with driver.session(database="neo4j") as session:
    session.execute_write(upload_providers, providers_csv_data)

In [14]:
with driver.session(database="neo4j") as session:
    session.execute_write(lambda tx: tx.run("""
        MATCH (pro:Provider), (e:Encounter)
        WHERE e.PROVIDER = pro.Id
        MERGE (e)-[:PROVIDED_BY]->(pro)
    """))

In [21]:
careplans_csv_data = []
with open("csv/careplans.csv", newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        careplans_csv_data.append(row)

In [22]:
def upload_careplans(tx, records):
    cypher_query = """
    UNWIND $records as record
    MERGE (care:Careplan {Id: record.Id})
    SET care += record
    WITH care, record
    MATCH (e:Encounter {Id: record.ENCOUNTER})
    MERGE (care)-[:FROM_ENCOUNTER]->(e)
    """
    tx.run(cypher_query, records=records)

with driver.session(database="neo4j") as session:
    session.execute_write(upload_careplans, careplans_csv_data)

In [23]:
procedures_csv_data = []
with open("csv/procedures.csv", newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        procedures_csv_data.append(row)

In [24]:
def upload_procedures(tx, records):
    cypher_query = """
    UNWIND $records as record
    CREATE (proc:Procedure)
    SET proc = record
    WITH proc, record
    MATCH (e:Encounter {Id: record.ENCOUNTER})
    MERGE (proc)-[:FROM_ENCOUNTER]->(e)
    """
    tx.run(cypher_query, records=records)

with driver.session(database="neo4j") as session:
    session.execute_write(upload_procedures, procedures_csv_data)

In [26]:
with driver.session(database="neo4j") as session:
    session.execute_write(lambda tx: tx.run("""
        MATCH (p:Patient), (m:Medication)
        WHERE p.Id = m.PATIENT
        MERGE (p)-[:HAS_MEDICATION]->(m)
    """))

In [27]:
payers_csv_data = []
with open("csv/payers.csv", newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        payers_csv_data.append(row)

In [30]:
def upload_payers(tx, records):
    cypher_query = """
    UNWIND $records as record
    MERGE (pay:Payer {Id: record.Id})
    SET pay += record
    WITH pay, record
    MATCH (e:Encounter {PAYER: record.Id})
    MERGE (e)-[:PAID_BY]->(pay)
    """
    tx.run(cypher_query, records=records)

with driver.session(database="neo4j") as session:
    session.execute_write(upload_payers, payers_csv_data)

In [33]:
claims_csv_data = []
with open("csv/claims.csv", newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        claims_csv_data.append(row)

In [34]:
def upload_claims(tx, records):
    cypher_query = """
    UNWIND $records as record
    MERGE (cl:Claim {Id: record.Id})
    SET cl += record
    WITH cl, record
    MATCH (p:Patient {Id: record.PATIENTID})
    MERGE (cl)-[:CLAIMED_BY]->(p)
    """
    tx.run(cypher_query, records=records)

with driver.session(database="neo4j") as session:
    session.execute_write(upload_claims, claims_csv_data)